In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import SparkSession

from shared.constants import DatasetPath

In [4]:
DATASET = DatasetPath('com-youtube')

In [5]:
spark = (SparkSession.builder
         .appName(f'{DATASET}')
         .config('spark.sql.legacy.timeParserPolicy', 'LEGACY')
         .config("spark.executor.memory", "8g")
         .config("spark.driver.memory", "8g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "16g")
         .getOrCreate())

22/01/03 00:08:24 WARN Utils: Your hostname, megatron resolves to a loopback address: 127.0.1.1; using 192.168.1.89 instead (on interface enp7s0)
22/01/03 00:08:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/03 00:08:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/03 00:08:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/01/03 00:08:26 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [17]:
schema = T.StructType([
    T.StructField('src', T.IntegerType(), False),
    T.StructField('dst', T.IntegerType(), False),
])

df = (
    spark.read.csv(DATASET.raw_str('com-youtube.ungraph.txt'), sep='\t', header=False, schema=schema, mode='DROPMALFORMED')
)
print(df.head(5))
print(df.count())

[Row(src=1, dst=2), Row(src=1, dst=3), Row(src=1, dst=4), Row(src=1, dst=5), Row(src=1, dst=6)]
2987628


In [18]:
df_nodes = (
    df.select(F.col('src').alias('id'))
        .union(df.select(F.col('dst').alias('id')))
        .distinct()
)
df_nodes.head(5)

[Row(id=148), Row(id=463), Row(id=471), Row(id=496), Row(id=1088)]

In [19]:
df_edges = (
    df.select(F.col('src'), F.col('dst'))
        .distinct()
)
df_edges.head(5)

[Row(src=2, dst=3386),
 Row(src=2, dst=3468),
 Row(src=2, dst=3696),
 Row(src=4, dst=5135),
 Row(src=4, dst=17394)]

In [21]:
df_nodes.write.parquet(DATASET.processed_str('nodes_User'), mode='overwrite')
df_edges.write.parquet(DATASET.processed_str('edges_FRIENDS'), mode='overwrite')

In [22]:
from datasets.build_schema import build_schema

build_schema(
    spark,
    name=str(DATASET),
    nodes=[
        ('User', DATASET.processed_str('nodes_User')),
    ],
    edges=[
        ('Friends', 'User', 'User', DATASET.processed_str('edges_FRIENDS')),
    ]
)

DatasetSchema(name='com-youtube', prefix='ComYoutube', database='com-youtube', description=None, nodes=[NodeSchema(label='User', path='data/processed/com-youtube/nodes_User', properties=[Property(name='id', type='int', ignore=False, label=False)])], edges=[EdgeSchema(type='FRIENDS', source='User', target='User', path='data/processed/com-youtube/edges_FRIENDS', properties=[Property(name='src', type='int', ignore=False, label=False), Property(name='dst', type='int', ignore=False, label=False)])])